In [1]:
import logging
import random
import time

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
def retry(attempts):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(attempts):
                try:
                    logging.info("Retry happening")
                    return func(*args, **kwargs)
                except Exception as e:
                    time.sleep(1)
                    logging.debug(e)
            return "Failure after all attempts"

        return wrapper

    return decorator

In [4]:
@retry(attempts=3)
def connect_to_database():
    if random.randint(0, 1):
        raise Exception("Temporary Database Error")
    return "Connected to Database"

In [5]:
for i in range(1, 6):
    logging.info(f"Connection attempt #{i}")
    print(f"--> {connect_to_database()}")

INFO:root:Connection attempt #1
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Connection attempt #2
INFO:root:Retry happening


--> Failure after all attempts


DEBUG:root:Temporary Database Error
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Connection attempt #3
INFO:root:Retry happening


--> Failure after all attempts


DEBUG:root:Temporary Database Error
INFO:root:Retry happening
DEBUG:root:Temporary Database Error
INFO:root:Retry happening
INFO:root:Connection attempt #4
INFO:root:Retry happening


--> Connected to Database


DEBUG:root:Temporary Database Error
INFO:root:Retry happening
INFO:root:Connection attempt #5
INFO:root:Retry happening


--> Connected to Database
--> Connected to Database


<hr>

In [8]:
!pip install tenacity

In [9]:
import logging
import random
import requests
import tenacity

In [10]:
logging.basicConfig(level=logging.DEBUG)

In [11]:
# The URL of the hypothetical API
API_URL = "http://localhost:5000/api"

In [12]:
@tenacity.retry(stop=tenacity.stop_after_attempt(3), wait=tenacity.wait_fixed(2))
def fetch_data_from_api():
    try:
        response = requests.get(API_URL)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()
    except requests.RequestException as e:
        raise RuntimeError(f"Failed to fetch data from the API: {str(e)}")

In [13]:
# Failed because server is not running
try:
    data = fetch_data_from_api()
    print("Data successfully fetched:", data)
except RuntimeError as e:
    print(f"Error: {str(e)}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000


RetryError: RetryError[<Future at 0x7d807c347c50 state=finished raised RuntimeError>]

In [14]:
# Success
try:
    data = fetch_data_from_api()
    print("Data successfully fetched:", data)
except RuntimeError as e:
    print(f"Error: {str(e)}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /api HTTP/1.1" 200 35


Data successfully fetched: {'msg': 'Welcome to our API!'}
